# EXPENSE TRACKER
**To use this application with users already created and therefore with expenses already registered in the database, run the code in the second cell and look at the printed credentials. Otherwise, create a new user and start entering your expenses and budgets!! Have fun!**


In [1]:
#IMPORT LIBRARIES

import random
import datetime
import calendar
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import customtkinter as ctk
from tkcalendar import DateEntry
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from collections import defaultdict
import matplotlib.dates as mdates


In [2]:
#CREATE A RANDOM DB / RUN THIS IF YOU WANT TO START WITH SOME RANDOM DATA ALREADY AVAILABLE.
#The usernames and passwords are going to be printed !!


category_options = ["Fuel", "Restaurant", "Grocery", "Rent", "Other"]

def random_date(start, end):
    return start + datetime.timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

notes_examples = ['Dinner with friends', 'Weekly groceries', 'Monthly rent', 'Fuel for road trip', 'Gym membership', 'Miscellaneous']


start_date = datetime.datetime(2022, 10, 1)
end_date = datetime.datetime.today()




def create_expenses():
    expenses = {}
    for i in range(1, 101):
        expenses[i] = {
            'expense': round(random.uniform(10, 500), 2),  
            'category': random.choice(category_options),
            'notes': random.choice(notes_examples),
            'date': random_date(start_date, end_date).strftime('%m/%d/%y')
        }
    return expenses
    
    
username_examples = ["Tommaso", "Madline", "Duarte", "Jacob"]
password_examples = ["1234", "NOVASBE", "Python", "Programming"]
    

def generate_random_budget():
    budget = {}
    for year in range(2020, 2024):  # 2020 to 2023
        for month in calendar.month_name[1:]:  # January to December
            budget[(str(year), month)] = round(random.uniform(1300, 2000),2)
    return budget



users = {}

for i in range(1, 4):
    expensesss = create_expenses()
    username = "User " + str(i)
    users[i] = {
        'username': username,
        'password': random.choice(password_examples),
        'expenses': expensesss,
        'budget': generate_random_budget(),
        'categories': ["Fuel", "Restaurant", "Grocery", "Rent", "Other"]
    }
    
    
print("3 users with data already inside\n")
for user in users:
    print("Username: {}".format(users[user]["username"]))
    print("Password: {}\n".format(users[user]["password"]))


3 users with data already inside

Username: User 1
Password: Programming

Username: User 2
Password: NOVASBE

Username: User 3
Password: 1234



In [3]:
#ALL THE FUNCTIONS ARE IN THIS CELL



#function to change page (frame)       

def raise_frame(frame):
    frame.tkraise()
    


#FROM NOW ON, FUNCTION FOR THE MAIN PAGE:


#function to add an expense to the expenses dictionary...
def add_expense(current_user_id, expense, category, notes, datee):
    
    if users[current_user_id]["expenses"]:
        last_id = max(users[current_user_id]["expenses"].keys())
    else:
        last_id = -1 

    if users[current_user_id]["expenses"]:
            last_id = max(users[current_user_id]["expenses"].keys())
    users[current_user_id]["expenses"][last_id+1] = {
        "expense": expense,
        "category": category,
        "notes": notes,
        "date" : datee
    }
    
#function for the Button "Save".
def SaveExpense():
    expense_value = expense_entry.get()
    category = category_menu.get()
    notes = expense_notes.get()
    date = date_entry.get()
    
    try:
        expense_value = float(expense_value) 
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter a valid number for the expense.")
        return  

    # Check if a category is selected
    if not category or category == "":  
        messagebox.showerror("No Category Selected", "Please select a category.")
        return 

    # Add the expense to the dictionary and then update the table to make sure the table will display the new expense
    add_expense(current_user_id, expense_value, category, notes, date)
    expense_entry.delete(0, tk.END)
    selected_category.set('')  
    expense_notes.delete(0, tk.END)
   
    current_date = datetime.datetime.now()
    date_entry.set_date(current_date)
    
    update_table()
    

#function to add a custom category
def Add_Category():
    new_cat = newcategory_entry.get()
    
    if not new_cat or new_cat == "":  
        messagebox.showerror("Error", "Please write a category.")
        return  
        
    print(current_user_id)
    users[current_user_id]["categories"].append(new_cat)
    newcategory_entry.delete(0, tk.END)
    
    update_category_menu()
    
#function to update the category menu, i use it after i add a new category (it does not update, but it deletes the button and it creates another one with the new category)
def update_category_menu():
    global category_menu
    category_menu.destroy()  
    category_menu = ctk.CTkOptionMenu(main_frame, variable=selected_category, values=users[current_user_id]["categories"], font=button_font, text_color=font_color, fg_color="#E8E8E8", button_color=button_color) 
    category_menu.grid(row=3, column=1, padx=10, pady=10, sticky="e")
  


 #update the table with all the expenses 
def update_table():
    
    sorted_expenses = sorted(users[current_user_id]["expenses"].items(), key=lambda x: datetime.datetime.strptime(x[1]['date'], '%m/%d/%y'), reverse = True)

   
    for i in expense_table.get_children():
        expense_table.delete(i)

   
    for expense_id, expense in sorted_expenses:
        expense_table.insert("", "end", iid=expense_id, values=(expense["expense"], expense["category"], expense["date"], expense["notes"]))
        

#function to delete the the expense selected in the table, both from the dictionary and from the table
def delete_selected_expense():
    selected_items = expense_table.selection()
    if not selected_items:
        messagebox.showwarning("Error", "Please select an item to delete.")
        return

    confirm = messagebox.askyesno("Confirm Delete", "Are you sure you want to delete this expense?")
    if confirm:
        for item in selected_items:
            
            try:
                expense_id = int(item)  
                del users[current_user_id]["expenses"][expense_id]  
                expense_table.delete(item)  
            except ValueError:
                messagebox.showerror("Error", "Invalid expense ID.")

            

    
# Function to add or update the monthly budget
def set_monthly_budget(current_user_id, year, month, budget):
    global monthly_budget_entry
    global month_combobox
    global year_combobox
    
    key = (year, month)

    
    
    if key in users[current_user_id]["budget"]:
         users[current_user_id]["budget"][key] += budget
    else:
        users[current_user_id]["budget"][key] = budget
  

    monthly_budget_entry.delete(0, tk.END)
    monthly_budget_entry = ctk.CTkEntry(main_frame, font=font_style, fg_color="#E8E8E8")
    monthly_budget_entry.grid(row=6, column=2, padx=10, pady=10, sticky="e")
    
    month_combobox.set('')  
    year_combobox.set('')

# Function to get the monthly budget
def get_monthly_budget(year, month):
    return users[current_user_id]["budget"].get((year, month), 0)


# Function to save the monthly budget
def save_monthly_budget():
    global monthly_budget_entry
    global month_combobox
    global year_combobox
    year = year_combobox.get()
    month = month_combobox.get()
    budget = monthly_budget_entry.get()
    key = (year, month)
   
    try:
        budget = float(budget)
        if budget <= 0:
            raise ValueError("Budget must be positive")
    except ValueError as e:
        messagebox.showerror("Invalid Input", f"Please enter a valid number for the budget.")
        
        return

    set_monthly_budget(current_user_id, year, month, budget)
    total_budget = get_monthly_budget(year, month)
    messagebox.showinfo("Success", f"Total budget for {month}/{year}: {total_budget}")
    
    
#function to log_out and go back to log in / sign-up page
def logout_function(frame):
    frame.tkraise()
    username_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)

    
#FROM NOW ON ALL THE FUNCTIONS FOR THE LOGIN / SIGN-UP PAGE    
    
current_user_id = 1   


def Login_Function():
    global current_user_id 
    
    username = username_entry.get()
    password = password_entry.get()
    
    if users:
            last_id = max(users.keys())
    
    for user_id, user_info in users.items():
        if user_info["username"] == username:
            if user_info["password"] == password:
                
                current_user_id = user_id
                update_table() 
                update_category_menu() #i put another time update table and update menu because, example: if I log in with user1, then log out, and then log in again with user 2, the main page shows me still the data of user 1
                raise_frame(main_frame) 
                break
            else:
                if user_id == last_id:
                    messagebox.showerror("Error", "Error: Your password or your username is wrong, try again")
                    password_entry.delete(0, tk.END)
                    username_entry.delete(0, tk.END)
                    break
        else:
            if user_id == last_id:
                messagebox.showerror("Error", "Error: Your password or your username is wrong, try again")
                password_entry.delete(0, tk.END)
                username_entry.delete(0, tk.END)
                break
                
        
def Signup_Function():
    username = new_username_entry.get()
    password = new_password_entry.get()
    password_2 = repeat_password_entry.get()
    for id, details in users.items():
        if username in details["username"]:
            messagebox.showerror("Error", "Username already exists")
            new_username_entry.delete(0, tk.END)
            new_password_entry.delete(0, tk.END)
            repeat_password_entry.delete(0, tk.END)
            return
    
    
    if password == password_2:
        if users:
            last_id = max(users.keys())
        else:
            last_id = -1 
        users[last_id + 1] = {
            'username': username,
            'password': password,
            'expenses': {},
            'budget': {},
            'categories': ["Fuel", "Restaurant", "Grocery", "Rent", "Other"]
        }
        new_username_entry.delete(0, tk.END)
        new_password_entry.delete(0, tk.END)
        repeat_password_entry.delete(0, tk.END)

    else:
        messagebox.showerror("Error", "Passwords do not match")




#FROM HERE ON THE CODE TO VISUALIZE THE GRAPH

def visualize_user_expensess(filtered_expenses, filtered_budget, charts_frame):
    
    #GRAPH0: SHOW THE BUDGET AND EXPENSES BY MONTH
    monthly_expenses = defaultdict(float)
    for expense in filtered_expenses.values():
        date = datetime.datetime.strptime(expense['date'], '%m/%d/%y')
        month_key = (str(date.year), calendar.month_name[date.month])
        monthly_expenses[month_key] += expense['expense']

    # Convert the month name to the month number for sorting
    month_number = {month: index for index, month in enumerate(calendar.month_name) if month}
    months_tuples = [(int(year), month_number[month]) for year, month in monthly_expenses.keys()]
    months_tuples += [(int(year), month_number[month]) for year, month in filtered_budget.keys() if (year, month) not in monthly_expenses]

 
    months_sorted = sorted(set(months_tuples), key=lambda x: (x[0], x[1]))

  
    months = [(str(year), calendar.month_name[month]) for year, month in months_sorted]

    expense_values = [monthly_expenses.get(month, 0) for month in months]
    budget_values = [filtered_budget.get(month, 0) for month in months]

  
    fig, ax = plt.subplots()
    bar_width = 0.35
    index = range(len(months))

    # Set the colors for expenses and budget
    expense_color = '#CC0000'
    budget_color = '#0090A0'

    bars1 = ax.bar(index, expense_values, bar_width, label='Expenses', color=expense_color)
    bars2 = ax.bar([i + bar_width for i in index], budget_values, bar_width, label='Budget', color=budget_color)

    ax.set_xlabel('Month')
    ax.set_ylabel('Amount')
    ax.set_title('Monthly Expenses and Budget Comparison')
    ax.set_xticks([i + bar_width / 2 for i in index])
    ax.set_xticklabels(months)
    ax.legend()

    plt.xticks(rotation=45)
    plt.tight_layout()

    canvas = FigureCanvasTkAgg(fig, master=charts_frame)
    canvas.draw()
    canvas.get_tk_widget().grid(row=0, column=0, sticky="nsew")

    df = pd.DataFrame(filtered_expenses).T
    df['expense'] = pd.to_numeric(df['expense'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    
    #GRAPH1: PIE CHART
    pie_colors = ['#96CDCD', '#0090A0', '#CC0000', '#FF6666', '#66CCCC', '#006666', '#FF9999', '#9999FF']
    fig1 = Figure(figsize=(5, 5))
    ax1 = fig1.add_subplot(111)
    category_expenses = df.groupby('category')['expense'].sum().sort_values(ascending=False)
    ax1.pie(category_expenses, labels=category_expenses.index, autopct='%1.1f%%', colors=pie_colors)
    ax1.set_title(f'Expense Distribution by Category for User')
    canvas1 = FigureCanvasTkAgg(fig1, master=charts_frame_visualization)
    canvas1.draw()
    canvas1.get_tk_widget().grid(row=1, column=1, sticky="ewn")
    
    # GRAPH 2: EXPENSES OVER TIME
    fig2 = Figure(figsize=(8, 4))
    ax2 = fig2.add_subplot(111)

    df_resampled = df.resample('M').sum(numeric_only=True)
    df_resampled.plot(kind='line', marker='o', ax=ax2, color = "#CC0000")

    ax2.set_title(f'Monthly Expenses Trend')
    ax2.set_ylabel('Total Expenses')
    ax2.set_xlabel('Month')

    canvas2 = FigureCanvasTkAgg(fig2, master=charts_frame_visualization)
    canvas2.draw()
    canvas2.get_tk_widget().grid(row=1, column=0, sticky="nsew")
    
    
    # SHOWING BALANCE
    total_expenses = sum(monthly_expenses.values())
    total_budgets = sum(filtered_budget.values())
    balance = total_budgets - total_expenses
    
    balance_label = ctk.CTkLabel(charts_frame_visualization, text=f"》Current Balance for the selected period: ${balance:.2f}", font=("Helvetica", 16), bg_color="white")

    # Set the text color based on the balance
    if balance >= 0:
        balance_label.configure(text_color="green")
    else:
        balance_label.configure(text_color="#CC0000")

    # Set the size of the label
    balance_label.configure(width=15, height=2)

    # Place the label in the frame
    balance_label.grid(row=0, column=1, sticky="nsw")
    

    
#FUNCTIONS TO FILTER THE DATA AND THEN GO TO VISUALIZATION

def filter_data_and_visualization():
    
    raise_frame(charts_frame_visualization)
    start_date = start_date_entry.get_date()
    end_date = end_date_entry.get_date()
    
    selected_indices = category_listbox.curselection()
    selected_categories = [category_listbox.get(i) for i in selected_indices]
    
    if selected_categories == []:
        selected_categories = users[current_user_id]["categories"]
    
    expenses = users[current_user_id]['expenses']
    budget = users[current_user_id]['budget']

    # Filter expenses based on the date range
    filtered_expenses = {}
    for expense_id, expense_data in expenses.items():
        expense_date = datetime.datetime.strptime(expense_data['date'], '%m/%d/%y').date()
        if (start_date <= expense_date <= end_date) and (expense_data['category'] in selected_categories):
            filtered_expenses[expense_id] = expense_data
            
    filtered_budget = {}
    for date_key, budget_amount in budget.items():
        year, month = date_key
        budget_date = datetime.datetime.strptime(f"{year}-{month}", '%Y-%B').date()
        # Set the day to the first of the month for comparison
        budget_date = budget_date.replace(day=1)
        if start_date <= budget_date <= end_date:
            filtered_budget[date_key] = budget_amount
    
            
    
    visualize_user_expensess(filtered_expenses, filtered_budget, charts_frame_visualization)


In [ ]:
#ALL THE CODE FOR THE INTERFACE AND DESIGN (APART FROM THE GRAPH THAT ARE IN THE "visualize_user_expensess" FUNCTION)

#create the Root
root = ctk.CTk(fg_color="#FFFFFF")
root.title("Expense Tracker")
root.geometry("700x600")

#main page
main_frame = ctk.CTkFrame(root)
main_frame.configure(fg_color="#FFFFFF")

#login page
login_frame = ctk.CTkFrame(root, fg_color="#FFFFFF")
login_frame.place(relwidth=0.5, relheight=0.5, relx=0.25, rely=0.25)

#charts filter page
charts_frame = ctk.CTkFrame(root)
charts_frame.configure(fg_color="#FFFFFF")

#charts visualization page
charts_frame_visualization = ctk.CTkFrame(root)
charts_frame_visualization.configure(fg_color="#FFFFFF")

for frame in (main_frame, charts_frame, login_frame, charts_frame_visualization):
    frame.grid(row=0, column=0, sticky='news')

# Styling
font_style = ("Helvetica", 13, "bold")
font_color = "#505050"
button_color = "#0090A0"
button_font_color = "#FFFFFF"
button_hover = "#007B7F"
button_font=("Helvetica", 13)
sub_font_style = ("Helvetica", 12)


# FROM NOW ON LOGIN AND SIGN-UP PAGE


# Configure grid for center alignment
login_frame.grid_columnconfigure(0, weight=1)
login_frame.grid_columnconfigure(1, weight=1)
login_frame.grid_columnconfigure(2, weight=1)

# Configure rows to expand equally
for i in range(10):  
    login_frame.grid_rowconfigure(i, weight=1)

# Login buttons and labels
username_label = ctk.CTkLabel(login_frame, text="Username", font=font_style, text_color=font_color)
username_label.grid(row=1, column=1,pady=4)

username_entry = ctk.CTkEntry(login_frame, corner_radius=8, fg_color="#E8E8E8")
username_entry.grid(row=1, column=2,padx=4, pady=4, sticky="w")

password_label = ctk.CTkLabel(login_frame, text="Password", font=font_style, text_color=font_color)
password_label.grid(row=2, column=1,pady=4)

password_entry = ctk.CTkEntry(login_frame, corner_radius=8, fg_color="#E8E8E8")
password_entry.grid(row=2, column=2, padx=4,pady=4, sticky="w")

login_button = ctk.CTkButton(login_frame, text="Log In", font=button_font, text_color=button_font_color, fg_color=button_color, hover_color=button_hover, command=Login_Function)
login_button.grid(row=3, column=2, pady=8, sticky="w")

# Signup buttons and labels
new_username_label = ctk.CTkLabel(login_frame, text="New Username", font=font_style, text_color=font_color)
new_username_label.grid(row=5, column=1, pady=4)

new_username_entry = ctk.CTkEntry(login_frame, corner_radius=8, fg_color="#E8E8E8")
new_username_entry.grid(row=5, column=2, padx=4, pady=4, sticky="w")

new_password_label = ctk.CTkLabel(login_frame, text="New Password", font=font_style, text_color=font_color)
new_password_label.grid(row=6, column=1, pady=4)

new_password_entry = ctk.CTkEntry(login_frame, corner_radius=8, fg_color="#E8E8E8")
new_password_entry.grid(row=6, column=2, padx=4, pady=4, sticky="w")

repeat_new_password_label = ctk.CTkLabel(login_frame, text="Repeat Password", font=font_style, text_color=font_color)
repeat_new_password_label.grid(row=7, column=1, pady=4)

repeat_password_entry = ctk.CTkEntry(login_frame, corner_radius=8, fg_color="#E8E8E8")
repeat_password_entry.grid(row=7, column=2, padx=4, pady=4, sticky="w")

signup_button = ctk.CTkButton(login_frame, text="Sign Up", font=button_font, text_color=button_font_color, fg_color=button_color, hover_color=button_hover, command=Signup_Function)
signup_button.grid(row=8, column=2, pady=8, sticky="w")




#FROM NOW ON MAIN PAGE


#expense label and entry
expense_label = ctk.CTkLabel(main_frame, text="Amount", font=font_style, text_color=font_color)
expense_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")

expense_entry = ctk.CTkEntry(main_frame, corner_radius=8, fg_color="#E8E8E8")
expense_entry.grid(row=0, column=1, padx=10, pady=10, sticky="w")

#Date label and entry
Date_label = ctk.CTkLabel(main_frame, text="Date", font=font_style, text_color=font_color)
Date_label.grid(row=1, column=0, padx=10, pady=10, sticky="w")

current_date = datetime.datetime.now()
date_entry = DateEntry(main_frame, font=button_font, text_color=font_color, year=current_date.year, month=current_date.month, day=current_date.day, width=13)
date_entry.grid(row=1, column=1, padx=10, pady=10, sticky="we")

#notes label end entry
notes_label = ctk.CTkLabel(main_frame, text="Notes", font=font_style, text_color=font_color)
notes_label.grid(row=2, column=0, padx=10, pady=10, sticky="w")

expense_notes = ctk.CTkEntry(main_frame,fg_color="#E8E8E8")
expense_notes.grid(row=2, column=1,columnspan=3, padx=10, pady=10, sticky="we")


#category label and options Menu 
category_label = ctk.CTkLabel(main_frame, text="Category", font=font_style, text_color=font_color)
category_label.grid(row=3, column=0, padx=10, pady=10, sticky="w")

selected_category = ctk.StringVar(root)
category_menu = ctk.CTkOptionMenu(main_frame, variable=selected_category, values=users[current_user_id]["categories"], font=button_font, text_color=font_color, fg_color="#E8E8E8", button_color=button_color)
category_menu.grid(row=3, column=1, padx=10, pady=10, sticky="w")


#SAVE BUTTON
submit_button = ctk.CTkButton(main_frame, text="Save", font=button_font, text_color=button_color, fg_color="#96CDCD", hover_color=button_hover, command=SaveExpense)
submit_button.grid(row=3, column=2, padx=10, pady=10, sticky="e")


#Entry and button for adding a new category
newcategory_label = ctk.CTkLabel(main_frame, text="Enter New Category ❯", font=sub_font_style, text_color=font_color)
newcategory_label.grid(row=4, column=1, padx=10, pady=10, sticky="e")

newcategory_entry = ctk.CTkEntry(main_frame, fg_color="#E8E8E8")
newcategory_entry.grid(row=4, column=2, padx=10, pady=10, sticky="e")
    
new_category_button = ctk.CTkButton(main_frame, text="Add New Category", font=button_font, text_color=button_font_color, fg_color=button_color, hover_color=button_hover, command=Add_Category)
new_category_button.grid(row=4, column=3, padx=10, pady=10, sticky="e")


# Entry and Lables for monthly budget
budget_label = ctk.CTkLabel(main_frame, text="Monthly Budget", font=font_style, text_color=font_color)
budget_label.grid(row=5, column=0, padx=10, pady=10, sticky="w")

timeframe_label = ctk.CTkLabel(main_frame, text="Select Month & Year ❯", font=sub_font_style, text_color=font_color)
timeframe_label.grid(row=5, column=1, padx=10, pady=10, sticky="e")

budgetamount_label = ctk.CTkLabel(main_frame, text="Enter Budget Amount ❯", font=sub_font_style, text_color=font_color)
budgetamount_label.grid(row=6, column=1, padx=10, pady=10, sticky="e")

monthly_budget_entry = ctk.CTkEntry(main_frame, font=font_style, fg_color="#E8E8E8")
monthly_budget_entry.grid(row=6, column=2, padx=10, pady=10, sticky="e")

# Dropdown for month and year
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = [str(year) for year in range(2020, 2031)]  # Example range

month_combobox = ttk.Combobox(main_frame, values=months, state="readonly", width=13)
month_combobox.grid(row=5, column=2, padx=10, pady=10, sticky="we")

year_combobox = ttk.Combobox(main_frame, values=years, state="readonly", width=13)
year_combobox.grid(row=5, column=3, padx=10, pady=10, sticky="we")

# Save button for monthly budget
save_budget_button = ctk.CTkButton(main_frame, text="Set Budget", font=button_font, text_color=button_font_color, fg_color=button_color,hover_color=button_hover, command=save_monthly_budget)
save_budget_button.grid(row=6, column=3, padx=10, pady=10, sticky="e")

# Create the table to display all the expenses
columns = ("Expense", "Category", "Date", "Notes")
expense_table = ttk.Treeview(main_frame, columns=columns, show="headings")

#set the column widhts
expense_table.column("Expense", width=150)
expense_table.column("Category", width=150)
expense_table.column("Date", width=150)
expense_table.column("Notes", width=250)

# Set the row height using style
style = ttk.Style()
style.configure("Treeview.Item", rowheight=40)

for col in columns:
    expense_table.heading(col, text=col)

# Create a scrollbar
scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=expense_table.yview)
expense_table.configure(yscroll=scrollbar.set)


expense_table.grid(row=7, column=0,columnspan=4, padx=(10, 0), sticky="nswe")
scrollbar.grid(row=7, column=4,padx=(0, 10), sticky="nsw")

update_table()


#Delete expense Button
delete_button = ctk.CTkButton(main_frame, text="Delete Selected Expense", font=button_font, text_color=button_font_color,command=delete_selected_expense,fg_color="#CC0000", hover_color="#B20000")#, width=350)
delete_button.grid(row=8, column=0, columnspan=2, padx=10, pady=10, sticky="w")

#Navigate Button
navigate_button = ctk.CTkButton(main_frame, text="Configure Charts ❯", font=button_font, text_color=button_font_color, fg_color=button_color, hover_color=button_hover, command=lambda: raise_frame(charts_frame)) #width=350)
navigate_button.grid(row=8, column=2, columnspan=2, padx=10, pady=10, sticky="e")


# Logout Button
log_out_button = ctk.CTkButton(main_frame, text="Log Out", font=button_font, text_color=button_font_color, fg_color="#D3D3D3",hover_color="#333333", command=lambda: logout_function(login_frame),width=100)
log_out_button.grid(row=9, column=0, padx=10, pady=10, sticky="w")


# FROM NOW ON CHARTS FILTER PAGE

# Start date entry
start_date_label = ctk.CTkLabel(charts_frame, text="Select Start Date",font=font_style, text_color=font_color)
start_date_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

start_date_entry = DateEntry(charts_frame, width=12, borderwidth=2, font=button_font, text_color=font_color)
start_date_entry.grid(row=0, column=1, padx=10, pady=10)

# End date entry
end_date_label = ctk.CTkLabel(charts_frame, text="Select End Date", font=font_style, text_color=font_color)
end_date_label.grid(row=1, column = 0, padx=10, pady=5, sticky="w")

end_date_entry = DateEntry(charts_frame, width=12, borderwidth=2, font=button_font, text_color=font_color)
end_date_entry.grid(row=1, column=1, padx=10, pady=10)

# Select Category Lable
select_category_label = ctk.CTkLabel(charts_frame, text="Select Categories ▼ ", font=font_style, text_color=font_color)
select_category_label.grid(row=2, column = 0, padx=10, pady=5, sticky="w")

# Button to select categories and move to Charts
select_button = ctk.CTkButton(charts_frame, text="Select and Display Charts ❯", font=font_style, text_color=button_font_color, fg_color=button_color, hover_color=button_hover, command=filter_data_and_visualization)#filter_data_and_visualization
select_button.grid(row=4, column=1, padx=10, pady=10, sticky="e")

#back to menu button
back_button = ctk.CTkButton(charts_frame, text="Back to Main Menu", font=font_style, text_color=button_font_color, fg_color="#96CDCD",hover_color=button_hover, command=lambda: raise_frame(main_frame))
back_button.grid(row=4, column=0, padx=10, pady=10, sticky="w")
categories = users.get(current_user_id, {}).get("categories", [])

# Create a Listbox for category selection
category_listbox = tk.Listbox(charts_frame, selectmode='multiple')
category_listbox.grid(row=3, column=0, columnspan=2, padx=(10, 0), sticky="we")
for category in categories:
    category_listbox.insert(tk.END, category)    
 


 #CHARTS VISUALIZATION PAGE (BUTTON TO GO BACK TO FILTERS)
back_to_filter_button = ctk.CTkButton(charts_frame_visualization, text="Back to Filters", font=font_style, text_color=button_font_color, fg_color="#96CDCD",hover_color=button_hover, command=lambda: raise_frame(charts_frame))
back_to_filter_button.grid(row=1, column=2, sticky="s")    
    
    
#starts the application with the login_page
raise_frame(login_frame)



root.mainloop()
